In [1]:
# import necessary items
# mostly copied over from MM_Covid_Explain2.ipynb
import re
import simdjson as sj
import gzip
from tqdm import tqdm
import pandas as pd
from tqdm.notebook import tqdm
import swifter
import seaborn as sns
import numpy as np	
from nltk import sent_tokenize
import re
from sentence_transformers import SentenceTransformer
from wutils.general import save_pickle, load_pickle
from wutils.mat import MarkedMatrix
from scipy.spatial.distance import cdist
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from treeinterpreter import treeinterpreter as ti

ModuleNotFoundError: No module named 'simdjson'

In [2]:
# import new library to use kernels
from sklearn.metrics.pairwise import pairwise_kernels

In [3]:
# used to time certain functions that take a long time
# not really needed, but somewhat interesting
import time

In [4]:
# load cord-19 (reference) data from pickle
# changed from ./pickles/who_cord_df.pkl to ./pickles/newest_cord_df_ft.pkl
cord_df = load_pickle('./pickles/newest_cord_df_ft.pkl')

In [5]:
# load good (true) data from pickle
# changed from ./pickles/newest_good_sample_df_ft.pkl to clean_good_df.pkl
good_df = load_pickle('./pickles/clean_good_df.pkl')

In [6]:
# load bad (false) data from pickle
# changed from ./pickles/newest_bad_df_ft.pkl to cleab_bad_df.pkl
bad_df = load_pickle('./pickles/clean_bad_df.pkl')

In [7]:
# example of good article
good_df[good_df.cleanText.swifter.apply(lambda x: 'Virus-ridden particles are inhaled by others and come into contact with cells' in x)]

Pandas Apply:   0%|          | 0/97539 [00:00<?, ?it/s]

,authors,publishDate,cleanText,title,url,covidFreq,sentences,fixed_sent,sent_embeddings,ft_embeddings,mean_embed,hostname,ft_has_nan
1206633,[Robin Mckie],2020-04-24 00:00:00,But their impact has been mild compared with t...,Coronavirus: what have scientists learned abou...,https://www.theguardian.com/world/2020/apr/24/...,13,[But their impact has been mild compared with ...,[But their impact has been mild compared with ...,"[[-0.15770671, -1.8202316, 0.33610916, -0.9967...","[[-0.020828864, 0.004317664, 0.000431179, 0.04...","[-0.007586088, -0.021697361, 0.0035677361, 0.0...",www.theguardian.com,False


In [8]:
# function to parse urls
import urllib.parse

def get_hostname(url):
    o = urllib.parse.urlsplit(url)
    return o.hostname

In [9]:
# function that returns true if text has to do with keep_if (wuhan in this case)
# returns false if text contains words in blacklist
# if text contains words from neither, defaults to true
def news_filter(text):
    keep_if = ['wuhan']
    text = text.lower()
    blacklist = ['u.s.', 'america', 'korea', 'china', 'mexico', 'australia', 'uk', 'u.k.', 'new york', 'los angeles', 'nfl', 'nba', 'mlb', 'epl']
    if any(x in text for x in keep_if):
        return True
    if any(x in text for x in blacklist):
        return False
    return True

In [10]:
# not really sure what any of the below does to be honest
# I assume it has to do with preparing the data somehow
good_df['hostname'] = good_df['url'].swifter.apply(get_hostname)
bad_df['hostname'] = bad_df['url'].swifter.apply(get_hostname)

Pandas Apply:   0%|          | 0/97539 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/21135 [00:00<?, ?it/s]

In [11]:
good_df['mean_embed'] = good_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))
bad_df['mean_embed'] = bad_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))

Pandas Apply:   0%|          | 0/97539 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/21135 [00:00<?, ?it/s]

In [12]:
# seems that we are keeping the bad articles only if they have this "is ap" quality
# I'm not sure what "is ap" means, but that's what this function does
is_AP = bad_df['cleanText'].swifter.apply(lambda x: '(AP)' in x)
print('Found: ', is_AP.sum())
bad_df = bad_df[~is_AP]

Pandas Apply:   0%|          | 0/21135 [00:00<?, ?it/s]

Found:  6268


In [13]:
# keyword
KW = 'transmission'

In [14]:
# don't know what this does
# more data prep I guess
cord_df = cord_df.dropna()
cord_df['mean_embed'] = cord_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))

Pandas Apply:   0%|          | 0/10149 [00:00<?, ?it/s]

In [15]:
# filtering dataframe to look for the keyword
# don't know why will named this "has_vax" if the keyword is transmission
# but I won't question it
has_vax = cord_df[cord_df.title.swifter.apply(lambda x: KW in x.lower()) | cord_df.abstract.swifter.apply(lambda x: KW in x.lower())]
vax_ref = np.vstack(has_vax['mean_embed'])

Pandas Apply:   0%|          | 0/10149 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/10149 [00:00<?, ?it/s]

## From the paper:

"We extract only articles that contain the word “transmission” and name it the “transmission news” dataset. The purpose ... is to provide a smaller sample with articles focusing more on attributes of the virus rather than on other topics."

The results for these smaller more specific datasets differs from the overall dataset as well, with the transmission news dataset seeming to fare a little bit worse than the other ones.

To show the effectiveness of his algorithm, Will is choosing to use the *"transmission news"* dataset. 

![table1](../../table_1.png)

In [16]:
# will commented this out
# vax_good_df = good_df[good_df['cleanText'].swifter.apply(lambda x: KW in x.lower())]
# vax_bad_df = bad_df[bad_df['cleanText'].swifter.apply(lambda x: KW in x.lower())]

# creating dataframes and cleaning them up
vax_good_df = good_df
vax_bad_df = bad_df

# applying news filter function:
# (removing text containing unwanted keywords, unless they also contain other wanted keywords)
vax_good_df = vax_good_df[vax_good_df['cleanText'].swifter.apply(news_filter)]
vax_bad_df = vax_bad_df[vax_bad_df['cleanText'].swifter.apply(news_filter)]

# turning this into a matrix I think
vax_good_mat = np.vstack(vax_good_df.mean_embed)
vax_bad_mat = np.vstack(vax_bad_df.mean_embed)

# turning matrix into a marked matrix (matrix with labels)
# will's own personal python library actually, impressive
vax_raw_mm = MarkedMatrix([('good', vax_good_mat), ('bad', vax_bad_mat)])

Pandas Apply:   0%|          | 0/97539 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/14867 [00:00<?, ?it/s]

In [19]:
# will calculates the distances between reference dataframe and to-be-labeled dataframe here I am assuming
# below is a link to possible other metrics to use besides cosine
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

# vgood2ref is the distance between vax_good_mat and vax_ref
vgood2ref = cdist(vax_good_mat, vax_ref, 'cosine')
print('done w/ good')

# vbad2ref is the distance between vax_bad_mat and vax_ref
vbad2ref = cdist(vax_bad_mat, vax_ref, 'cosine')
print('done w/ bad')

# turning this into a marked matrix
vax_mm = MarkedMatrix([('good', vgood2ref), ('bad', vbad2ref)])

# theoretically, vgood2ref should contain small distances (since good articles should be similar to CORD-19)
# and vbad2ref should contain large distances (since bad articles should not be similar to CORD-19)

done w/ good
done w/ bad


In [19]:
# Alt method
# commented out by Will originally
# has_vax = cord_df.sample(5000)
# vax_ref = np.vstack(has_vax['mean_embed'])

In [21]:
# War on Terror Cells: Strategies to Eradicate "Novel Coronavirus" Effectively
np.where(~np.isfinite(vax_ref))

(array([], dtype=int64), array([], dtype=int64))

In [22]:
# initializing random forest classifiers to be used later
# random forest: a collection of decision trees, where the final decision is the average of all the trees
# the idea is: the crowd is smarter
raw_tmodel = RandomForestClassifier(n_estimators=500, n_jobs=28)
tmodel = RandomForestClassifier(n_estimators=500, n_jobs=28)
tmodel2 = RandomForestClassifier(n_estimators=1500, n_jobs=28)

In [25]:
# finding some examples of good text
tidx = 263
vax_good_df.iloc[tidx].title, vax_good_df.iloc[tidx].cleanText

('Prices of alcohols used in making hand sanitizers capped under Essential Commodities Act',
 'The government on Thursday said prices of alcohols used in manufacturing hand sanitizers have been capped under the Essential Commodities Act on account of coranavirus outbreak.The decision would empower the central government and states/union territories to regulate prices, production, sale, distribution, transport, movement, storage, information of alcohols used in manufacturing hand sanitizers, used as preventive measure to avoid infection from COVID-19 , Ministry of Consumer Affairs said in a statement.It would also empower governments to smoothen the sale and availability of these items and carry out operations against hoarders, speculators, profiteers, black marketeers and those involved in contravention of the orders of central government and states/UTs in respect of these alcohols.It will help manufacturers of hand sanitizers to keep prices of their product at reasonable level and wit

In [26]:
# function written by will to show program's classification decision process for individual documents
def tree_explain_good(idx):
    pred, bias, contrib = ti.predict(tmodel, vgood2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', vax_good_df.iloc[idx].cleanText)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_bad = np.argsort(cmat[:, 0])[::-1]
    import_good = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

# pretty much a copy of the previous function, except used to compare with bad dataset rather than good
def tree_explain_bad(idx):
    pred, bias, contrib = ti.predict(tmodel, vbad2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', vax_bad_df.iloc[idx].cleanText)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_bad = np.argsort(cmat[:, 0])[::-1]
    import_good = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

In [27]:
# this was commented out by will
# vax_mm.mat[~np.isfinite(vax_mm.mat)] = 0

**Training and evaluation begins after this point.**

*Quick Refresher:*
* vax_mm was defined above as a marked matrix of vgood2ref as good, and vbad2ref as bad
* vgood2ref is the matrix of distances between the good matrix and the vax ref
* vbad2ref is the matrix of distances between the bad matrix and the vax ref
* ideally, vgood2ref should have small distances
* while vbad2ref whould have large distances

**Note:** *single_split_classify* is a function from will's wutils python library.

The source can be found here: https://github.com/willshiao/wutils/blob/master/wutils/mat.py

The numbers that single_split_classify returns are in the following format: ((accuracy, f1-score), labels)

In [37]:
# results from running program on vax_raw_mm
rawResults = dict()

In [28]:
# ====== RUN FIRST TIME WITH COSINE ======
cosineResults = dict()

In [31]:
# run vax_mm with tmodel
print('trial_1:')
start = time.time()
res = vax_mm.single_split_classify(tmodel, return_labels=True)
print(res)
cosineResults['trial_1'] = res
end = time.time()
print('seconds: ', end - start)

trial_1:
Training model...
Evaluating model...
((0.9103275460891355, 0.9516689176152807), {'bad': 0, 'good': 1})
seconds:  76.65508675575256


In [32]:
# running vax_mm with tmodel again
print('trial_2:')
start = time.time()
res = vax_mm.single_split_classify(tmodel, return_labels=True)
print(res)
cosineResults['trial_2'] = res
end = time.time()
print('seconds: ', end - start)

trial_2:
Training model...
Evaluating model...
((0.9133703239663504, 0.9535775944753501), {'bad': 0, 'good': 1})
seconds:  79.59276962280273


**Important Note:** 

trial_3 is a bit different from the other trials.

Unlike the other trials, trial_3 uses *vax_raw_mm* instead of *vax_mm.* vax_raw_mm contains the absolute positions of each article instead of the distances from each article to the collection of reference articles. 

After running the below code, I was able to obtain an accuracy of roughly 0.92 and f1-score of roughly 0.96. I have no idea how the results can be so good with just the absolute positions of the documents and not the distances, (since the distances are what determine how likely it is that each document is "good" or "bad"), but I suppose that this is a question for Will.

Most importantly though, when we are comparing the effectiveness of different kernels or different distance calculation equations, we should ignore this trial.

In [33]:
# running vax_raw_mm with raw_t_model
print('trial_3:')
start = time.time()
res = vax_raw_mm.single_split_classify(raw_tmodel, return_labels=True)
print(res)
rawResults['trial_3'] = res
end = time.time()
print('seconds: ', end - start)

trial_3:
Training model...
Evaluating model...
((0.9205298013245033, 0.9574549635875814), {'bad': 0, 'good': 1})
seconds:  72.37524795532227


In [34]:
# running vax_mm with tmodel2
print('trial_4:')
start = time.time()
res = vax_mm.single_split_classify(tmodel2, return_labels=True)
print(res)
cosineResults['trial_4'] = res
end = time.time()
print('seconds: ', end - start)

trial_4:
Training model...
Evaluating model...
((0.9126543762305351, 0.9530949634755864), {'bad': 0, 'good': 1})
seconds:  232.05667996406555


**trial_5 also uses vax_raw_mm and so should also be ignored when comparing the effectiveness of different kernels/distance calculations.**

In [35]:
# running vax_raw_mm with tmodel2
print('trial_5:')
start = time.time()
res = vax_raw_mm.single_split_classify(tmodel2, return_labels=True)
print(res)
rawResults['trial_5'] = res
end = time.time()
print('seconds: ', end - start)

trial_5:
Training model...
Evaluating model...
((0.9115804546268123, 0.9524862941233048), {'bad': 0, 'good': 1})
seconds:  217.21354818344116


In [46]:
# print the contents of cosineResults
for e in cosineResults:
    print('===================')
    print(e)
    print('accuracy: ', cosineResults[e][0][0])
    print('f1-score: ', cosineResults[e][0][1])
    print('labels: ', cosineResults[e][1])

trial_1
accuracy:  0.9103275460891355
f1-score:  0.9516689176152807
labels:  {'bad': 0, 'good': 1}
trial_2
accuracy:  0.9133703239663504
f1-score:  0.9535775944753501
labels:  {'bad': 0, 'good': 1}
trial_4
accuracy:  0.9126543762305351
f1-score:  0.9530949634755864
labels:  {'bad': 0, 'good': 1}


**Changing from a simple distance calculation to a kernel after this point.**

From this point on, we are going to be trying out a bunch of kernels to see if they work better than a simple cosine distance calculation. I don't really fully understand all the math behind how these kernels work, but now we're about to see if they are better.

The documentation for pairwise_kernels states that "If the input is a kernel matrix, it is returned instead." While we are using vax_good_mat as a matrix, it is technically a Python array data structure, and so it won't get automatically returned. pairwise_kernel will indeed perform an operation on it.

In [21]:
# documentation for pairwise_kernels found here:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_kernels.html
# I don't think that the order of which matrix is first matters, but we can test with different arrangements
# possible kernels: [‘additive_chi2’, ‘chi2’, ‘linear’, ‘poly’, ‘polynomial’, ‘rbf’, ‘laplacian’, ‘sigmoid’, ‘cosine’]

vgood2ref = pairwise_kernels(vax_good_mat, vax_ref, metric='polynomial')
print('done w/ good')

vbad2ref = pairwise_kernels(vax_bad_mat, vax_ref, metric='polynomial')
print('done w/ bad')

vax_mm = MarkedMatrix([('good', vgood2ref), ('bad', vbad2ref)])

done w/ good
done w/ bad


In [44]:
# ====== RUN WITH POLYNOMIAL KERNEL ======
polyKernelResults = dict()

In [45]:
print('trial_1:')
start = time.time()
res = vax_mm.single_split_classify(tmodel, return_labels=True)
print(res)
polyKernelResults['trial_1'] = res
end = time.time()
print('seconds: ', end - start)

trial_1:
Training model...
Evaluating model...
((0.9097905852872741, 0.9514076359429232), {'bad': 0, 'good': 1})
seconds:  58.91968321800232


In [47]:
# running vax_mm with tmodel again
print('trial_2:')
start = time.time()
res = vax_mm.single_split_classify(tmodel, return_labels=True)
print(res)
polyKernelResults['trial_2'] = res
end = time.time()
print('seconds: ', end - start)

trial_2:
Training model...
Evaluating model...
((0.9081797028816896, 0.9505446833124459), {'bad': 0, 'good': 1})
seconds:  61.162139892578125


In [62]:
# skip trial_3 and trial_5 since those use vax_raw_mm
# running vax_mm with tmodel2
print('trial_4:')
start = time.time()
res = vax_mm.single_split_classify(tmodel2, return_labels=True)
print(res)
polyKernelResults['trial_4'] = res
end = time.time()
print('seconds: ', end - start)

trial_4:
Training model...
Evaluating model...
((0.9088956506175049, 0.9510906120880177), {'bad': 0, 'good': 1})
seconds:  178.65489196777344


**Side Note:**

Not the information I was looking for originally, but it seems that using a kernel reduces the training + evaluation time by a noticable amount, with the time reduction being proportional to the total time taken.

In [49]:
# print the contents of polyKernelResults
for e in polyKernelResults:
    print('===================')
    print(e)
    print('accuracy: ', polyKernelResults[e][0][0])
    print('f1-score: ', polyKernelResults[e][0][1])
    print('labels: ', polyKernelResults[e][1])

trial_1
accuracy:  0.9097905852872741
f1-score:  0.9514076359429232
labels:  {'bad': 0, 'good': 1}
trial_2
accuracy:  0.9081797028816896
f1-score:  0.9505446833124459
labels:  {'bad': 0, 'good': 1}
trial_4
accuracy:  0.9104170395561124
f1-score:  0.9517660097335325
labels:  {'bad': 0, 'good': 1}


In [52]:
# compare results of polyKernel with results of cosine:
print('Note: positive difference means that poly-kernel is better.')
for e in polyKernelResults:
    print('===================')
    print(e)
    print('poly-kernel accuracy: ', polyKernelResults[e][0][0])
    print('cosine accuracy: ', cosineResults[e][0][0])
    print('difference: ', polyKernelResults[e][0][0] - cosineResults[e][0][0])
    print('poly-kernel f1-score: ', polyKernelResults[e][0][1])
    print('cosine f1-score: ', cosineResults[e][0][1])
    print('difference: ', polyKernelResults[e][0][1] - cosineResults[e][0][1])
    print('labels: ', polyKernelResults[e][1])

Note: positive difference means that poly-kernel is better.
trial_1
poly-kernel accuracy:  0.9097905852872741
cosine accuracy:  0.9103275460891355
difference:  -0.0005369608018613858
poly-kernel f1-score:  0.9514076359429232
cosine f1-score:  0.9516689176152807
difference:  -0.0002612816723575051
labels:  {'bad': 0, 'good': 1}
trial_2
poly-kernel accuracy:  0.9081797028816896
cosine accuracy:  0.9133703239663504
difference:  -0.00519062108466084
poly-kernel f1-score:  0.9505446833124459
cosine f1-score:  0.9535775944753501
difference:  -0.0030329111629041616
labels:  {'bad': 0, 'good': 1}
trial_4
poly-kernel accuracy:  0.9104170395561124
cosine accuracy:  0.9126543762305351
difference:  -0.0022373366744227186
poly-kernel f1-score:  0.9517660097335325
cosine f1-score:  0.9530949634755864
difference:  -0.0013289537420538844
labels:  {'bad': 0, 'good': 1}


**Polynomial kernel vs cosine function analysis:**

It seems that there isn't really a noticable improvement from using a polynomial kernel vs using a simple cosine equation. In fact, in all 3 cases, the cosine function actually happened to be slightly better, although the difference is negligible.

In [53]:
# ====== RUN WITH RBF KERNEL ======
rbfKernelResults = dict()

In [63]:
vgood2ref = pairwise_kernels(vax_good_mat, vax_ref, metric='rbf')
print('done w/ good')

vbad2ref = pairwise_kernels(vax_bad_mat, vax_ref, metric='rbf')
print('done w/ bad')

vax_mm = MarkedMatrix([('good', vgood2ref), ('bad', vbad2ref)])

done w/ good
done w/ bad


In [55]:
print('trial_1:')
start = time.time()
res = vax_mm.single_split_classify(tmodel, return_labels=True)
print(res)
rbfKernelResults['trial_1'] = res
end = time.time()
print('seconds: ', end - start)

trial_1:
Training model...
Evaluating model...
((0.9072847682119205, 0.9499613601236476), {'bad': 0, 'good': 1})
seconds:  75.75212001800537


In [56]:
# running vax_mm with tmodel again
print('trial_2:')
start = time.time()
res = vax_mm.single_split_classify(tmodel, return_labels=True)
print(res)
rbfKernelResults['trial_2'] = res
end = time.time()
print('seconds: ', end - start)

trial_2:
Training model...
Evaluating model...
((0.9119384284947198, 0.9527241279907754), {'bad': 0, 'good': 1})
seconds:  75.78960728645325


In [64]:
# skip trial_3 and trial_5 since those use vax_raw_mm
# running vax_mm with tmodel2
print('trial_4:')
start = time.time()
res = vax_mm.single_split_classify(tmodel2, return_labels=True)
print(res)
rbfKernelResults['trial_4'] = res
end = time.time()
print('seconds: ', end - start)

trial_4:
Training model...
Evaluating model...
((0.9110434938249508, 0.9521701472428062), {'bad': 0, 'good': 1})
seconds:  228.27178812026978


In [65]:
# print the contents of rbfKernelResults
for e in rbfKernelResults:
    print('===================')
    print(e)
    print('accuracy: ', rbfKernelResults[e][0][0])
    print('f1-score: ', rbfKernelResults[e][0][1])
    print('labels: ', rbfKernelResults[e][1])

trial_1
accuracy:  0.9072847682119205
f1-score:  0.9499613601236476
labels:  {'bad': 0, 'good': 1}
trial_2
accuracy:  0.9119384284947198
f1-score:  0.9527241279907754
labels:  {'bad': 0, 'good': 1}
trial_4
accuracy:  0.9110434938249508
f1-score:  0.9521701472428062
labels:  {'bad': 0, 'good': 1}


In [68]:
# compare results of rbfKernel with results of cosine:
print('Note: positive difference means that rbf-kernel is better.')
for e in rbfKernelResults:
    print('===================')
    print(e)
    print('rbf-kernel accuracy: ', rbfKernelResults[e][0][0])
    print('cosine accuracy: ', cosineResults[e][0][0])
    print('difference: ', rbfKernelResults[e][0][0] - cosineResults[e][0][0])
    print('rbf-kernel f1-score: ', rbfKernelResults[e][0][1])
    print('cosine f1-score: ', cosineResults[e][0][1])
    print('difference: ', rbfKernelResults[e][0][1] - cosineResults[e][0][1])
    print('labels: ', rbfKernelResults[e][1])

Note: positive difference means that rbf-kernel is better.
trial_1
rbf-kernel accuracy:  0.9072847682119205
cosine accuracy:  0.9103275460891355
difference:  -0.003042777877214964
rbf-kernel f1-score:  0.9499613601236476
cosine f1-score:  0.9516689176152807
difference:  -0.0017075574916330938
labels:  {'bad': 0, 'good': 1}
trial_2
rbf-kernel accuracy:  0.9119384284947198
cosine accuracy:  0.9133703239663504
difference:  -0.0014318954716305843
rbf-kernel f1-score:  0.9527241279907754
cosine f1-score:  0.9535775944753501
difference:  -0.0008534664845746276
labels:  {'bad': 0, 'good': 1}
trial_4
rbf-kernel accuracy:  0.9110434938249508
cosine accuracy:  0.9126543762305351
difference:  -0.0016108824055843796
rbf-kernel f1-score:  0.9521701472428062
cosine f1-score:  0.9530949634755864
difference:  -0.000924816232780179
labels:  {'bad': 0, 'good': 1}


**RBF kernel vs cosine function analysis:**

It seems that once again, there isn't really a noticable improvement from using an rbf kernel vs using a simple cosine equation. Furthermore, once again in all 3 cases, the cosine function actually happened to be slightly better, although the difference is again negligible.

In [69]:
# ====== RUN WITH LAPLACIAN KERNEL ======
laplacianKernelResults = dict()

In [70]:
vgood2ref = pairwise_kernels(vax_good_mat, vax_ref, metric='laplacian')
print('done w/ good')

vbad2ref = pairwise_kernels(vax_bad_mat, vax_ref, metric='laplacian')
print('done w/ bad')

vax_mm = MarkedMatrix([('good', vgood2ref), ('bad', vbad2ref)])

done w/ good
done w/ bad


In [71]:
print('trial_1:')
start = time.time()
res = vax_mm.single_split_classify(tmodel, return_labels=True)
print(res)
laplacianKernelResults['trial_1'] = res
end = time.time()
print('seconds: ', end - start)

trial_1:
Training model...
Evaluating model...
((0.912833363164489, 0.9532180595581172), {'bad': 0, 'good': 1})
seconds:  80.60169792175293


In [72]:
# running vax_mm with tmodel again
print('trial_2:')
start = time.time()
res = vax_mm.single_split_classify(tmodel, return_labels=True)
print(res)
laplacianKernelResults['trial_2'] = res
end = time.time()
print('seconds: ', end - start)

trial_2:
Training model...
Evaluating model...
((0.9153391802398425, 0.954510482785151), {'bad': 0, 'good': 1})
seconds:  77.38251423835754


In [73]:
# skip trial_3 and trial_5 since those use vax_raw_mm
# running vax_mm with tmodel2
print('trial_4:')
start = time.time()
res = vax_mm.single_split_classify(tmodel2, return_labels=True)
print(res)
laplacianKernelResults['trial_4'] = res
end = time.time()
print('seconds: ', end - start)

trial_4:
Training model...
Evaluating model...
((0.9104170395561124, 0.9518680578929655), {'bad': 0, 'good': 1})
seconds:  231.2023479938507


In [74]:
# print the contents of laplacianKernelResults
for e in laplacianKernelResults:
    print('===================')
    print(e)
    print('accuracy: ', laplacianKernelResults[e][0][0])
    print('f1-score: ', laplacianKernelResults[e][0][1])
    print('labels: ', laplacianKernelResults[e][1])

trial_1
accuracy:  0.912833363164489
f1-score:  0.9532180595581172
labels:  {'bad': 0, 'good': 1}
trial_2
accuracy:  0.9153391802398425
f1-score:  0.954510482785151
labels:  {'bad': 0, 'good': 1}
trial_4
accuracy:  0.9104170395561124
f1-score:  0.9518680578929655
labels:  {'bad': 0, 'good': 1}


In [75]:
# compare results of laplacianKernel with results of cosine:
print('Note: positive difference means that rbf-kernel is better.')
for e in laplacianKernelResults:
    print('===================')
    print(e)
    print('laplacian-kernel accuracy: ', laplacianKernelResults[e][0][0])
    print('cosine accuracy: ', cosineResults[e][0][0])
    print('difference: ', laplacianKernelResults[e][0][0] - cosineResults[e][0][0])
    print('laplacian-kernel f1-score: ', laplacianKernelResults[e][0][1])
    print('cosine f1-score: ', cosineResults[e][0][1])
    print('difference: ', laplacianKernelResults[e][0][1] - cosineResults[e][0][1])
    print('labels: ', laplacianKernelResults[e][1])

Note: positive difference means that rbf-kernel is better.
trial_1
laplacian-kernel accuracy:  0.912833363164489
cosine accuracy:  0.9103275460891355
difference:  0.002505817075353578
laplacian-kernel f1-score:  0.9532180595581172
cosine f1-score:  0.9516689176152807
difference:  0.0015491419428365072
labels:  {'bad': 0, 'good': 1}
trial_2
laplacian-kernel accuracy:  0.9153391802398425
cosine accuracy:  0.9133703239663504
difference:  0.001968856273492081
laplacian-kernel f1-score:  0.954510482785151
cosine f1-score:  0.9535775944753501
difference:  0.0009328883098009833
labels:  {'bad': 0, 'good': 1}
trial_4
laplacian-kernel accuracy:  0.9104170395561124
cosine accuracy:  0.9126543762305351
difference:  -0.0022373366744227186
laplacian-kernel f1-score:  0.9518680578929655
cosine f1-score:  0.9530949634755864
difference:  -0.0012269055826209518
labels:  {'bad': 0, 'good': 1}


**laplacian kernel vs cosine function analysis:**

It seems that for two out of the three trials, the laplacian kernel actually managed to do better than the cosine function, although only slightly. However, for the third trial, the laplacian kernel fared worse, however the difference for all three trials is once again negligable.

One thing that is interesting to note is that trial_1 and trial_2 use the same tmodel/Random Forest Classifier, one which contains 500 estimators. However, trial_4 uses a different Random Forest Classifier, tmodel2, which contains 1500 estimators. It seems that the laplacian kernel fared better when the number of estimators was lower however fared worse when the number of estimators was higher. 

I tried testing with a larger Random Forest Classifier (3000 estimators) but it took an ungodly amount of time to train, to the point where my Jupyter Lab would keep timing out and crashing, so I decided to just scrap the idea.